In [1]:
import pandas as pd
import json
import re
import warnings
warnings.filterwarnings('ignore')

Reference Function

In [2]:
#1.
def have_all_values_in_common(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    return set1 == set2
#2.
def is_list_included(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    return set1.issubset(set2)
#3.
def remove_special_characters(string):
    pattern = r'\W+'
    processed_string = re.sub(pattern, '', string)
    return processed_string
#4.
def remove_special_characters_2(string):
    pattern = r'\W+'
    processed_string = re.sub(pattern, ' ', string)
    return processed_string
#5.
def has_common_elements(list1, list2,threshold):
    common_count = 0
    for element in list1:
        if element in list2:
            common_count += 1
            if common_count > threshold:
                return True
    return False
#6.
def remove_repetitive_words(string):
    words = string.split()
    unique_words = [] 
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return ' '.join(unique_words)
#7.
def remove_text_after_last_hyphen(string):
    pattern = r'^(.*?)(?!.*eg)-.*$'
    match = re.match(pattern, string)
    if match:
        return match.group(1)
    else:
        return string
#8.
def has_common_pattern(strings_list):
    pattern = None
    for string in strings_list:
        if pattern is None:
            pattern = string
        else:
            match = re.search(pattern, string)
            if match is None:
                return False
    return True
#9.
def add_whitespace(string):
    pattern = r'(\D)(\d)|(\d)(\D)'
    modified_string = re.sub(pattern, r'\1 \2\3 \4', string)
    return modified_string
#10.
def remove_er(string):
    words = string.split()  
    modified_words = [word[:-2] if word.endswith("er") else word for word in words]
    return " ".join(modified_words)  


In [3]:
dataframe_RBN=pd.read_csv("Reference_Banks.csv")
dataframe_RBN["Bank_Name"]=dataframe_RBN["Bank_Name"].apply(lambda x:x.strip())
reference_banks_main=list(dataframe_RBN["Bank_Name"].unique())
dataframe_RBN.head()

,Bank_Name
0,Citibank Europe plc Frankfurt-Deutschland
1,886 GmbH
2,Aachener Bank eG
3,Aachener Bausparkasse AG
4,Landeskreditbank Baden-Württemberg


In [4]:
file_path = 'Extra_Banks.json'
with open(file_path, 'r') as file:
    dictionary = json.load(file)
Banks_adjusted=dictionary["Banks_Adjusted"]
Original_Banks=dictionary["Original_Banks"]

In [5]:
dataframe_zip_codes=pd.read_csv('German_postal_codes.csv',delimiter=',')
dataframe_zip_codes.head()

,PLZ,Ort
0,54298,Aach
1,78267,Aach
2,52062,Aachen
3,52064,Aachen
4,52066,Aachen


1. Bank Response Data

In [6]:
#Read Bank Response Data
dataframe=pd.read_excel('Bank Response Data.xlsx')
#Drop id column if it exists
try:
    dataframe.drop(columns=['id'],inplace=True)
except:
    pass
#Fix PLZ column
dataframe['PLZ']=dataframe['PLZ'].astype(str)
dataframe['PLZ']=dataframe['PLZ'].apply(lambda x:'0'+str(x) if len(x)==4 else x)
#Add Status Column
dataframe['Status']=' '
#Print
dataframe=pd.merge(dataframe,dataframe_zip_codes,on='PLZ',how='left')
#Dataframe 2
dataframe_2=dataframe[dataframe['Ort'].isna()]
#dataframe_2.to_csv('1.Invalid_PLZ_City_Match.csv',index=False)
#Dataframe 1
dataframe_1=dataframe[~dataframe['Ort'].isna()]

In [7]:
#Modify Original Bank Colum
dataframe_1['Bank']=dataframe_1['Bank'].astype(str)
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Zweigniederlassung Deutschland', '')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Niederlassung Deutschland', '')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('-Deutschland', '')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('KSK ', 'Kreissparkasse ')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('PB', 'Postbank')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('-Tuttlingen', '')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('– eine Niederlassung der Deutsche', '')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Commerzbak', 'Commerzbank')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Köln Bonn', 'KölnBonn')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Volksbanken', 'Volksbank')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Bayr', 'Bayer')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('VB', 'Volksbank')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('Weser', 'Wesser')
dataframe_1['Bank']=dataframe_1['Bank'].str.replace('J. P.', 'J.P.')
#Modify Adjusted Bank Column
#set all words to be lower case
dataframe_1['Bank Adjusted'] = dataframe_1['Bank'].apply(lambda x:x.lower())
#Remove special characters
dataframe_1['Bank Adjusted'] = dataframe_1['Bank Adjusted'].apply(lambda x:re.sub('\W+',' ',x))
#1.
dataframe_1['Bank Adjusted'] = dataframe_1['Bank Adjusted'].str.replace('kreisparkasse', 'kreissparkasse')
dataframe_1['Bank Adjusted'] = dataframe_1['Bank Adjusted'].str.replace('kreis sparkasse', 'kreissparkasse')
#2.
dataframe['Bank Adjusted'] = dataframe_1['Bank Adjusted'].str.replace('postalisch', '')
dataframe['Bank Adjusted'] = dataframe_1['Bank Adjusted'].str.replace('poststr', '')
#3.
dataframe_1['Bank Adjusted'] = dataframe_1['Bank Adjusted'].str.replace('lb bw', 'lbbw')


In [8]:
dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparkasse'),"Bank Adjusted"]=dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparkasse'),"Bank Adjusted"]+" "+dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparkasse'),"Ort"].apply(lambda x:x.lower())
dataframe_1.loc[dataframe_1['Bank'].str.contains('Stadtsparkasse'),"Bank Adjusted"]=dataframe_1.loc[dataframe_1['Bank'].str.contains('Stadtsparkasse'),"Bank Adjusted"]+" "+dataframe_1.loc[dataframe_1['Bank'].str.contains('Stadtsparkasse'),"Ort"].apply(lambda x:x.lower())
dataframe_1.loc[dataframe_1['Bank'].str.contains('Kreissparkasse'),"Bank Adjusted"]=dataframe_1.loc[dataframe_1['Bank'].str.contains('Kreissparkasse'),"Bank Adjusted"]+" "+dataframe_1.loc[dataframe_1['Bank'].str.contains('Kreissparkasse'),"Ort"].apply(lambda x:x.lower())
dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda-Bank'),"Bank Adjusted"]=dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda-Bank'),"Bank Adjusted"]+" "+dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda-Bank'),"Ort"].apply(lambda x:x.lower())
dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda Bank'),"Bank Adjusted"]=dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda Bank'),"Bank Adjusted"]+" "+dataframe_1.loc[dataframe_1['Bank'].str.contains('Sparda Bank'),"Ort"].apply(lambda x:x.lower())
dataframe_1["Bank Adjusted"]=dataframe_1["Bank Adjusted"].astype(str)
dataframe_1["Bank Adjusted"]=dataframe_1["Bank Adjusted"].apply(lambda x:remove_repetitive_words(x))
#6.
dataframe_1['Bank Adjusted'] = dataframe_1['Bank Adjusted'].apply(lambda x:x.strip())

In [9]:
dataframe_1=dataframe_1[['status','Bank','Bank Adjusted','Street','PLZ','Ort','Country','E-Mail','Response','Days to Respond','Sent Via','Status']]

In [10]:
print("The total number of rows are :",dataframe_1.shape[0])

The total number of rows are : 13495


In [11]:
for keys,values in Banks_adjusted.items():
    for keywords in values:
        list_of_indexes=[]
        for keyword in keywords:
            list_of_indexes.append(list(dataframe_1.loc[(dataframe_1['Bank Adjusted'].str.contains(keyword))&(dataframe_1['Status']!='Done')].index))
        list_of_indexes=list(set.intersection(*[set(list) for list in list_of_indexes]))
        dataframe_1.loc[list_of_indexes,'Bank Adjusted']=keys
        dataframe_1.loc[list_of_indexes,'Status']="Done"

In [12]:
for keys,values in Original_Banks.items():
    for keywords in values:
        list_of_indexes=[]
        for keyword in keywords:
            list_of_indexes.append(list(dataframe_1.loc[(dataframe_1['Bank'].str.contains(keyword))&(dataframe_1['Status']!='Done')].index))
        list_of_indexes=list(set.intersection(*[set(list) for list in list_of_indexes]))
        dataframe_1.loc[list_of_indexes,'Bank Adjusted']=keys
        dataframe_1.loc[list_of_indexes,'Status']="Done"

In [13]:
#Unique Bank Names in Original File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
#Reference Bank Names in Reference File
reference_banks_main=list(dataframe_RBN["Bank_Name"].unique())
#Drop If Status is Done
#Iterate over Unique Bank Names
for bank in  unique_bank_names:
        bank_split=bank.split(' ')
        for reference_bank in reference_banks_main:
                reference_bank_split=reference_bank.lower().split(' ')
                #d.Reference Bank Name and Adjusted Bank Name contain exact same vocabulary but in different order
                if(have_all_values_in_common(bank_split,reference_bank_split)==True):
                        filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                        dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank
                        dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done"
                        break

In [14]:
#Unique Bank Names in Original File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
#Drop If Status is Done
filteration_condition_2= (dataframe_1['Status']!="Done")
#Iterate over Unique Bank Names
for bank in  unique_bank_names:
        bank_checker=remove_special_characters(bank).strip()
        #Iterate over Reference Bank
        for reference_bank in reference_banks_main:
            #a. Reference Bank Name and Adjusted Bank Name have the same character pattern after removal of all special characters and setting it in lower case
            reference_bank_checker=remove_special_characters(reference_bank.lower()).strip()
            if(bank_checker==reference_bank_checker):
                filteration_condition_1=dataframe_1['Bank Adjusted']==bank
                dataframe_1.loc[(filteration_condition_1)&(filteration_condition_2),"Bank Adjusted"]=reference_bank
                dataframe_1.loc[(filteration_condition_1)&(filteration_condition_2),"Status"]="Done"
                break

In [15]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
for bank in  unique_bank_names:
        bank_split=bank.split(' ')
        counter=0
        for reference_bank in reference_banks_main:
                reference_bank_split=remove_special_characters_2(reference_bank).lower().split(" ")
                #a.Adjusted Bank Name is part of Reference Bank Name
                if(is_list_included(bank_split, reference_bank_split)):
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done"    

In [16]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
for bank in  unique_bank_names:
        counter=0
        for reference_bank in reference_banks_main:
                #a.Adjusted Bank Name is part of Reference Bank Name
                if bank in remove_special_characters_2(reference_bank.lower()):
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done" 

In [17]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
for bank in  unique_bank_names:
        for reference_bank in reference_banks_main:
                #a.Adjusted Bank Name is part of Reference Bank Name
                if reference_bank.lower() in bank:
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done" 

In [18]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
for bank in  unique_bank_names:
        bank_split=bank.split(' ')
        counter=0
        for reference_bank in reference_banks_main:
                reference_bank_split=remove_special_characters_2(reference_bank).lower().split(" ")
                #a.Adjusted Bank Name is part of Reference Bank Name
                if(is_list_included(reference_bank_split,bank_split)):
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done"    

In [19]:
#Reference Bank Names in Reference File
threshold_values=[1,2,3,4,5,6]
for threshold in threshold_values:
        unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
        for bank in  unique_bank_names:
                bank_split=bank.replace("bank","").split(" ")
                counter=0
                for reference_bank in reference_banks_main:
                        reference_bank_split=remove_special_characters_2(reference_bank).lower().replace("bank","").split(" ")
                        if(has_common_elements(reference_bank_split,bank_split,threshold)):
                                reference_bank_temp=reference_bank
                                counter=counter+1
                if(counter==1):
                        filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                        dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                        dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done"    

In [20]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank Adjusted'].unique())
for bank in  unique_bank_names:
        bank_checker=bank.lower()
        bank_checker=bank_checker.replace("stadtsparkasse","sparkasse").replace("kreissparkasse","sparkasse")
        bank_checker=bank_checker.split(" ")
        bank_checker=[s for s in bank_checker if s]
        counter=0
        for reference_bank in reference_banks_main:
                reference_bank_checker=remove_special_characters_2(reference_bank).lower()
                reference_bank_checker=reference_bank.replace("stadtsparkasse","sparkasse").replace("kreissparkasse","sparkasse")
                reference_bank_checker=reference_bank_checker.split(" ")
                reference_bank_checker=[s for s in reference_bank_checker if s]
                #a.Adjusted Bank Name is part of Reference Bank Name
                if(has_common_elements(bank_checker,reference_bank_checker,1)):
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank Adjusted']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done"    

In [21]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank'].unique())
for bank in  unique_bank_names:
        counter=0
        for reference_bank in reference_banks_main:
                #a.Adjusted Bank Name is part of Reference Bank Name
                if reference_bank in bank:
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank']==bank)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done" 

In [22]:
#Reference Bank Names in Reference File
unique_bank_names=list(dataframe_1[(dataframe_1['Status']!="Done")]['Bank'].unique())
for bank in  unique_bank_names:
        counter=0
        for bank in reference_bank:
                #a.Adjusted Bank Name is part of Reference Bank Name
                if bank in reference_bank:
                        reference_bank_temp=reference_bank
                        counter=counter+1
        if(counter==1):
                filteration_condition_1=(dataframe_1['Bank']==bank)
                print("Reference Bank",reference_bank_temp)
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
                dataframe_1.loc[(filteration_condition_1)&(dataframe_1['Status']!="Done"),"Status"]="Done" 

In [23]:
# Adjust Volksbank that are greater than 1
Bank_names=dataframe_1[(dataframe_1['Status']!='Done')]
Bank_names["Bank"] = Bank_names["Bank"].apply(lambda x:remove_text_after_last_hyphen(x))
Bank_names["Bank"] = Bank_names["Bank"].apply(lambda x:x.strip())
#Take only Metrics that have more than one values and less than one value, check if they have as match and take them
temp=Bank_names["Bank"].value_counts().reset_index()
for bank in temp["index"].values:
    counter=0
    for reference_bank in reference_banks_main:
        if bank in reference_bank:
            reference_bank_temp=reference_bank
            counter=counter+1
    if(counter==1):
        dataframe_1.loc[(dataframe_1['Bank'].str.contains(bank))&(dataframe_1['Status']!="Done"),"Bank Adjusted"]=reference_bank_temp
        dataframe_1.loc[(dataframe_1['Bank'].str.contains(bank))&(dataframe_1['Status']!="Done"),"Status"]="Done"        

In [24]:
file_path = 'OverdriveFile.json'
with open(file_path, 'r') as file:
    dictionary = json.load(file)
Original_Banks_overdrive=dictionary["Original_Banks"]

In [25]:
for keys,values in Original_Banks_overdrive.items():
    for keywords in values:
        list_of_indexes=[]
        for keyword in keywords:
            list_of_indexes.append(list(dataframe_1.loc[(dataframe_1['Bank'].str.contains(keyword))].index))
        list_of_indexes=list(set.intersection(*[set(list) for list in list_of_indexes]))
        dataframe_1.loc[list_of_indexes,'Bank Adjusted']=keys
        dataframe_1.loc[list_of_indexes,'Status']="Done"

In [26]:
dataframe_1[dataframe_1["Status"]=="Done"].shape[0]

13026

In [27]:
to_be_removed=["Bausparkasse der Volksbank und Raiffeisenbanken" , "Rheinland-Pfalz Bank", "SIKB/MI Mittelstands-Invest GmbH", "Ethikbank", "NatWest Markets -Frankfurt","Beschwingt und Froh Bank-Charlies Schokoladenfabrik","Bank Julius Bär Deutschland AG","KINTO","IBM","CACEIS","MAC"]
dataframe_1_done=dataframe_1[(dataframe_1["Status"]=="Done")&(~dataframe_1["Bank"].isin(to_be_removed))]
dataframe_1_not_done=dataframe_1[(dataframe_1["Status"]!="Done")|(dataframe_1["Bank"].isin(to_be_removed))]

In [28]:
#Fix Problems
dataframe_1_done.loc[dataframe_1_done["Bank"].str.lower()=="deutsche hypothekenbank","Bank Adjusted"]="Deutsche Hypothekenbank"
dataframe_1_done.loc[dataframe_1_done["Bank"]=="Raiffeisenbank","Bank Adjusted"]="Raiffeisenbank eG"
dataframe_1_done = dataframe_1_done.drop(12188)

In [29]:
#dataframe_1_not_done.to_csv("Not_Done.csv",index=False)

In [30]:
temp=dataframe_1_done[["Bank","Bank Adjusted"]].value_counts().reset_index()
temp.rename(columns={0:"Count"},inplace=True)
temp.to_csv("3.Bank_and_Corresponding_Match_latest.csv",index=False)

In [31]:
dataframe_1_done

,status,Bank,Bank Adjusted,Street,PLZ,Ort,Country,E-Mail,Response,Days to Respond,Sent Via,Status
0,received,Berliner Volksbank eG,Berliner Volksbank eG,Wittestraße 30 R,13509,Berlin,de,gundula.hoffmann@berliner-volksbank.de,True,40.0,postal,Done
1,waiting_signatures,Sparkasse Saarbrücken,Sparkasse Saarbrücken,Neumarkt 17,66117,Saarbrücken,de,Paul.Fluhr@sk-sb.de,True,NaN,postal,Done
2,received,Sparkasse Ansbach,Sparkasse Ansbach,Promenade 20,91522,Ansbach,de,dieter.bradler@sparkasse-ansbach.de,True,24.0,email,Done
3,waiting_confirmation,VR Bank Enz plus eG,VR Bank Enz plus eG,Hauptstr. 64,75196,Remchingen,de,NaN,True,NaN,postal,Done
4,waiting_signatures,Deutsche Bank AG,Deutsche Bank,Theodor-Heuss-Straße 3,70174,Stuttgart,de,monika.wick@db.com,False,NaN,email,Done
...,...,...,...,...,...,...,...,...,...,...,...,...
14134,waiting_signatures,"Commerzbank AG, Germany-München",Commerzbank AG,Landsberger Strasse 344,80687,München,de,NaN,False,NaN,postal,Done
14135,waiting_signatures,JPMorgan-Frankfurt,J.P. Morgan AG,Taunustor 1,60310,Frankfurt,de,Global.Audit.Confirmations@jpmchase.com,False,NaN,email,Done
14136,waiting_signatures,"Commerzbank AG, Germany-München",Commerzbank AG,Landsberger Strasse 344,80687,München,de,NaN,False,NaN,postal,Done
14137,ready_to_send,BNP Paribas-,BNP Paribas S.A.,Stanford Frankfurt: Europa-Allee 12,60327,Frankfurt,de,csd_germany@bnpparibas.com,False,NaN,email,Done


In [32]:
dataframe_1_not_done

,status,Bank,Bank Adjusted,Street,PLZ,Ort,Country,E-Mail,Response,Days to Respond,Sent Via,Status
59,accepted,Würth Leasing GmbH & Co. KG,würth leasing gmbh co kg,Frankfurter Straße 151,63303,Dreieich,de,karoline.belle@wuerth-truck-lease.de,True,77.0,email,
103,received,Sparkasse Augsburg,sparkasse augsburg,Halderstr. 1,86150,Augsburg,de,Christian.Proell@sska.de,True,25.0,email,
107,accepted,meine Volksbank Raiffeisenbank eG,meine volksbank raiffeisenbank eg,Gabelsbergerstr. 10,83308,Trostberg,de,manfred.holzner@vb-rb.de,True,22.0,email,
117,sent,Landesbausparkasse Südwest,landesbausparkasse südwest,Jägerstr. 46,70174,Stuttgart,de,NaN,False,NaN,postal,
127,accepted,VR-Bank Fulda,vr bank fulda,Bahnhofstraße 1,36037,Fulda,de,NaN,True,9.0,postal,
...,...,...,...,...,...,...,...,...,...,...,...,...
13854,sent,Amex Kreditkarte-DEUTDEFFXXX,amex kreditkarte deutdeffxxx,Theodor-Heuss-Allee 112,60486,Frankfurt,de,NaN,False,NaN,postal,
13995,accepted,Baden-Württembergische Bank-Stuttgart,baden württembergische bank stuttgart,Kleiner Schlossplatz 11,70173,Stuttgart,de,thomas.masthoff@bw-bank.de,True,5.0,postal,
14058,accepted,Bayern-Versicherung Lebensversicherung-Aktieng...,bayern versicherung lebensversicherung aktieng...,Am Karlsbad 4-5,10785,Berlin,de,insolvenzabsicherung@feuersozietaet.de,True,1.0,email,
14079,sent,Hannoversche Beteiligungsgesellschaft Niedersa...,hannoversche beteiligungsgesellschaft niedersa...,Schiffgraben 10,30159,Hannover,de,NaN,False,NaN,postal,


In [33]:
dataframe_1=dataframe_1_done
#Keep the following Status : Recieved, accepted, sent, reminded, rejected.
Status_to_be_kept=['received','accepted','sent','reminded','rejected']
dataframe_1=dataframe_1[dataframe_1['status'].isin(Status_to_be_kept)]
#E-mail
dataframe_1['E-Mail']=dataframe_1['E-Mail'].fillna('No E-mail')
dataframe_1['Street']=dataframe_1['Street'].fillna('')
#
dataframe_1['Number of requests']=0
dataframe_1.head()

,status,Bank,Bank Adjusted,Street,PLZ,Ort,Country,E-Mail,Response,Days to Respond,Sent Via,Status,Number of requests
0,received,Berliner Volksbank eG,Berliner Volksbank eG,Wittestraße 30 R,13509,Berlin,de,gundula.hoffmann@berliner-volksbank.de,True,40.0,postal,Done,0
2,received,Sparkasse Ansbach,Sparkasse Ansbach,Promenade 20,91522,Ansbach,de,dieter.bradler@sparkasse-ansbach.de,True,24.0,email,Done,0
5,accepted,Sparkasse,Sparkasse Nürnberg,Lorenzer Platz 12,90402,Nürnberg,de,harald.tirschner@sparkasse-nuernberg.de,True,7.0,postal,Done,0
6,received,HSBC Trinkaus & Burkhardt AG,HSBC Trinkaus & Burkhardt AG,Hansaallee 3,40549,Düsseldorf,de,kerstin.herwald@hsbc.de,True,6.0,email,Done,0
7,accepted,Volksbank Bocholt eG,Volksbank Bocholt eG,Postfach 2462,46374,Bocholt,de,No E-mail,True,9.0,postal,Done,0


In [44]:
part_1=dataframe_1.groupby(['Bank Adjusted','PLZ','Ort']).agg({'Street':list}).reset_index()
part_1["Street"]=part_1["Street"].apply(lambda x:[re.sub(r'(\d)\s+(\d)', r'\1\2', str(temp)) for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[string for string in x if any(c.isalpha() for c in string) and len(string) > 1])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("/"," ") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("  "," ") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("str.","straße") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("Strasse","Straße") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("pl.","plaz") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("Pl.","Plaz") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).title() for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("Str.","Straße") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("strasse","straße") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("Po Box","Postfach") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("alage","anlage") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace(" - ","-") for temp in x])
part_1["Street"]=part_1["Street"].apply(lambda x:[str(temp).replace("ße","ße ") for temp in x])
part_1['Street']=part_1['Street'].apply(lambda x:list(set(x)))
part_1["Street"]=part_1["Street"].apply(lambda x:sorted(x, key=len))
indexes_under_consideration=list(part_1[part_1["Street"].apply(lambda x:len(x))>1].index)
for x in indexes_under_consideration:
    temp=part_1.loc[x]
    Different_address =temp["Street"]
    for address_index in range(0,len(Different_address)):
        try:
            if(Different_address[address_index] in Different_address[address_index+1]):
                Different_address[address_index] = Different_address[address_index+1] 
        except:
            pass
    part_1.loc[x,"Street"]= Different_address
part_1['Street']=part_1['Street'].apply(lambda x:list(set(x)))
part_1["Street"]=part_1["Street"].apply(lambda x:sorted(x, key=len))

In [45]:
part_1=part_1.explode(["Street"])
part_1["Address"]=part_1[["Street","PLZ","Ort"]].values.tolist()
part_1.drop(columns=["Street"],inplace=True)
part_1=part_1.groupby(["Bank Adjusted","PLZ","Ort"]).agg({"Address":list}).reset_index()
part_1

,Bank Adjusted,PLZ,Ort,Address
0,ABN AMRO Bank N.V.,50670,Köln,"[[Gereonstraße 15-23, 50670, Köln]]"
1,ABN AMRO Bank N.V.,60320,Frankfurt,"[[Mainzer Landstraße 1, 60320, Frankfurt]]"
2,ABN AMRO Bank N.V.,60325,Frankfurt,"[[Ulmenstraße 23-25, 60325, Frankfurt]]"
3,ABN AMRO Bank N.V.,60329,Frankfurt,"[[Mainzer Landstraße 1, 60329, Frankfurt], [Ma..."
4,AGCO FinanceGbH,40549,Düsseldorf,"[[Theo-Champion-Straße 1, 40549, Düsseldorf]]"
...,...,...,...,...
2607,levoBank eG,66822,Lebach,"[[Poststraße 1, 66822, Lebach]]"
2608,mediserv Bank GmbH,66121,Saarbrücken,"[[Am Halberg 6, 66121, Saarbrücken]]"
2609,solarisBank AG,10178,Berlin,"[[Anna-Louis-Karsch-Straße 2, 10178, Berlin],..."
2610,solarisBank AG,10235,Berlin,"[[Warschauer Platz 11-13, 10235, Berlin]]"


In [46]:
part_2=dataframe_1.groupby(['Bank Adjusted','PLZ','Ort']).agg({'E-Mail':list,'Number of requests':'count'}).reset_index()
#E-mail
part_2['E-Mail']=part_2['E-Mail'].apply(lambda x:[temp for temp in x if temp != 'No E-mail'])
part_2['E-Mail']=part_2['E-Mail'].apply(lambda x:list(set(x)))
#
part_2

,Bank Adjusted,PLZ,Ort,E-Mail,Number of requests
0,ABN AMRO Bank N.V.,50670,Köln,[],6
1,ABN AMRO Bank N.V.,60320,Frankfurt,[henry.upmeyer@de.abnamro.com],1
2,ABN AMRO Bank N.V.,60325,Frankfurt,[kristin.sivinski@abnamroclearing.com],2
3,ABN AMRO Bank N.V.,60329,Frankfurt,"[Commercial.Banking@de.abnamro.com, paivi.borr...",8
4,AGCO FinanceGbH,40549,Düsseldorf,[georg.boentrup@agcofinance.com],2
...,...,...,...,...,...
2607,levoBank eG,66822,Lebach,"[thorsten.warken@levo-bank.de, info@levo-bank....",4
2608,mediserv Bank GmbH,66121,Saarbrücken,[francisco.diaz@de.ey.com],2
2609,solarisBank AG,10178,Berlin,"[info@Solarisbank.de, christian.kuehn@solarisb...",4
2610,solarisBank AG,10235,Berlin,[help@getpenta.com],2


In [47]:
#Check Bank PLZ Combination
Bank_PLZ_combination=part_2[['Bank Adjusted','PLZ']].drop_duplicates(keep='first')
#Response Time and Rate
Response_time=[]
Response_Rate=[]
#Postal Response Time and Rate
Postal_Response_time=[]
Postal_Response_rate=[]
#Email Response Time and Rate
Email_Response_time=[]
Email_Response_rate=[]
for bank,plz in Bank_PLZ_combination.values:
    temp=dataframe_1[(dataframe_1['Bank Adjusted']==bank)&(dataframe_1['PLZ']==plz)]
    #Overall Response Time and Rate
    Response_time.append(temp[temp['Response']==True]['Days to Respond'].mean())
    Response_Rate.append(temp[temp['Response']==True].shape[0]/temp.shape[0])
    #Email Response Time and Rate
    Email_Response_time.append(temp[(temp['Response']==True)&(temp['Sent Via']=='email')]['Days to Respond'].mean())
    try:
        Email_Response_rate.append(round(temp[(temp['Response']==True)&(temp['Sent Via']=='email')].shape[0]/temp[(temp['Sent Via']=='email')].shape[0],2))
    except:
        Email_Response_rate.append(None)

    #Postal Response Time and Rate
    Postal_Response_time.append(temp[(temp['Response']==True)&(temp['Sent Via']=='postal')]['Days to Respond'].mean())
    try:
        Postal_Response_rate.append(round(temp[(temp['Response']==True)&(temp['Sent Via']=='postal')].shape[0]/temp[(temp['Sent Via']=='postal')].shape[0],2))
    except:
        Postal_Response_rate.append(None)

In [48]:
#Overall Response
part_2['Overall Response Time']=Response_time
part_2['Overall Response Time']=part_2['Overall Response Time'].apply(lambda x:round(x,2))
part_2['Overall Response Rate']=Response_Rate
part_2['Overall Response Rate']=part_2['Overall Response Rate'].apply(lambda x:round(x,2))

In [49]:
#Email Response Time
part_2['E-mail Response Time']=Email_Response_time
part_2['E-mail Response Time']=part_2['E-mail Response Time'].apply(lambda x:round(x,2))
part_2['E-mail Response Rate']=Email_Response_rate
part_2['E-mail Response Rate']=part_2['E-mail Response Rate'].apply(lambda x:round(x,2))

In [50]:
#Postal Response Time
part_2['Postal Response Time']=Postal_Response_time
part_2['Postal Response Time']=part_2['Postal Response Time'].apply(lambda x:round(x,2))
part_2['Postal Response Rate']=Postal_Response_rate
part_2['Postal Response Rate']=part_2['Postal Response Rate'].apply(lambda x:round(x,2))

In [59]:
total=pd.merge(part_2,part_1,on=["Bank Adjusted","PLZ","Ort"],how="left")
total=total[['Bank Adjusted','Address',"PLZ","Ort",'E-Mail','Number of requests','Overall Response Time','Overall Response Rate','E-mail Response Time','E-mail Response Rate','Postal Response Time','Postal Response Rate']]
total["E-Mail"]=total["E-Mail"].apply(lambda x:[temp for temp in x if "@" in temp])
total

,Bank Adjusted,Address,PLZ,Ort,E-Mail,Number of requests,Overall Response Time,Overall Response Rate,E-mail Response Time,E-mail Response Rate,Postal Response Time,Postal Response Rate
0,ABN AMRO Bank N.V.,"[[Gereonstraße 15-23, 50670, Köln]]",50670,Köln,[],6,10.00,1.00,NaN,NaN,10.00,1.0
1,ABN AMRO Bank N.V.,"[[Mainzer Landstraße 1, 60320, Frankfurt]]",60320,Frankfurt,[henry.upmeyer@de.abnamro.com],1,12.00,1.00,12.0,1.00,NaN,NaN
2,ABN AMRO Bank N.V.,"[[Ulmenstraße 23-25, 60325, Frankfurt]]",60325,Frankfurt,[kristin.sivinski@abnamroclearing.com],2,55.00,1.00,49.0,1.00,61.00,1.0
3,ABN AMRO Bank N.V.,"[[Mainzer Landstraße 1, 60329, Frankfurt], [Ma...",60329,Frankfurt,"[Commercial.Banking@de.abnamro.com, paivi.borr...",8,37.20,0.62,16.0,0.67,51.33,0.6
4,AGCO FinanceGbH,"[[Theo-Champion-Straße 1, 40549, Düsseldorf]]",40549,Düsseldorf,[georg.boentrup@agcofinance.com],2,12.50,1.00,NaN,NaN,12.50,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2607,levoBank eG,"[[Poststraße 1, 66822, Lebach]]",66822,Lebach,"[thorsten.warken@levo-bank.de, info@levo-bank....",4,26.33,0.75,29.0,0.67,21.00,1.0
2608,mediserv Bank GmbH,"[[Am Halberg 6, 66121, Saarbrücken]]",66121,Saarbrücken,[francisco.diaz@de.ey.com],2,42.00,0.50,NaN,0.00,42.00,1.0
2609,solarisBank AG,"[[Anna-Louis-Karsch-Straße 2, 10178, Berlin],...",10178,Berlin,"[info@Solarisbank.de, christian.kuehn@solarisb...",4,10.33,0.75,NaN,0.00,10.33,1.0
2610,solarisBank AG,"[[Warschauer Platz 11-13, 10235, Berlin]]",10235,Berlin,[help@getpenta.com],2,35.50,1.00,35.5,1.00,NaN,NaN


In [60]:
total[["Bank Adjusted"]].value_counts()

Bank Adjusted                                                                                
Commerzbank AG                                                                                   209
Deutsche Bank                                                                                    132
HypoVereinsbank (Unicredit Bank)                                                                  72
UniCredit Group                                                                                   59
Deutsche Postbank AG                                                                              58
                                                                                                ... 
Sparkasse Gera-Greiz                                                                               1
Sparkasse Geseke                                                                                   1
Sparkasse Gevelsberg-Wetter (Ruhr) - Zweckverbandssparkasse der StädteGvelsberg Wetter (Ruhr)     

In [61]:
total.to_csv("4.Normalized_Data.csv",index=False)